In [9]:
%%capture
import sys
!{sys.executable} -m pip install requests pandas geopy bs4 folium

In [13]:
import folium
import requests
import pandas as pd
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import json

In [4]:
CLIENT_ID = 'DEPKYDHCHC40DUDYBECM3Y1GYBGOFGRTODGR4QLNQ5VPNLBU'
CLIENT_SECRET = 'FWKDHU4URB1WHQI5IKQ5QN4RMLEWOASA34AHLW3YNA4KCGH5'
VERSION = '20190808'

In [16]:
geolocator = Nominatim(user_agent="coursera agent")
def create_areas_data_frame(): 
    with open('area_index.html') as f:
        soup = BeautifulSoup(f, parser='lxml')

    city_areas = soup.find_all('div', attrs={'class': 'city-listing'})
    areas_dict = {'Area': [], 'Latitude': [], 'Longitude': []}
    for areas in city_areas:
        area_cat = areas.find('p', attrs={'class': 'key-values'})
        for area_parsed in area_cat.find_all('a'):
            area_short = area_parsed.text.split(',')[0]
            area = '{}, Pune'.format(area_short)
            try:
                location = geolocator.geocode(area)
                latitude = location.latitude
                longitude = location.longitude
            except Exception as e:
                pass
            else:
                areas_dict['Area'].append(area_short)
                areas_dict['Latitude'].append(latitude)
                areas_dict['Longitude'].append(longitude)

    df_areas = pd.DataFrame(areas_dict)
    # df_areas.to_csv('pune_areas.csv')
    return df_areas

def search_venues(latitude, longitude):
    limit = 50
    radius = 500
    base_url = 'https://api.foursquare.com/v2/venues/'
    endpoint = 'search'
    parameters = '?client_id={}&client_secret={}&v={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION)
    additional_params = '&limit={}&radius={}&ll={},{}'.format(limit, radius, latitude, longitude)
    url = '{}{}{}{}'.format(base_url, endpoint, parameters, additional_params)
    response = requests.get(url).json()
    return response

In [73]:
df_pune = pd.read_csv('pune_areas.csv')
df_pune.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
df_pune = df_pune[(df_pune['Latitude'] < 18.6) & (df_pune['Latitude'] > 18.35)
                  & (df_pune['Longitude'] < 74.0) & (df_pune['Longitude'] > 73.6)]
print(df_pune.shape)
df_pune.head(10)

(69, 3)


,Area,Latitude,Longitude
0,Aundh,18.561883,73.810196
3,Anand Nagar,18.478490,73.821326
5,Balewadi,18.582027,73.768983
6,Baner,18.564243,73.776857
7,Bund Garden Road,18.533404,73.878625
10,Budhwar Peth,18.517560,73.858037
11,Baner,18.564243,73.776857
12,Central Pune,18.520306,73.854319
17,Dhole Patil Road,18.534908,73.876258
18,Deccan Gymkhana,18.515893,73.841160


In [75]:
location = geolocator.geocode('Pune')
pune_map = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)
for lat, lon, area in zip(df_pune['Latitude'], df_pune['Longitude'], df_pune['Area']):
    label = folium.Popup(area)
    folium.CircleMarker(
        [lat, lon],
        color='blue',
        popup=label,
        radius=8,
        fill_color='yellow',
        fill=True,
        fill_opacity=0.8
    ).add_to(pune_map)
pune_map